# **Satellite Derived Bathymetry without sample elevation data by Google Earth Engine**

การทำแบบจำลองความลึกท้องน้ำตื้นโดยใช้ภาพถ่ายดาวเทียม Sentinel 2 (L2A : Bottom of the atmosphere) แปลงค่าการสะท้อนในช่วงคลื่นสีน้ำเงินและเขียวเป็นความลึกของน้ำผ่านสมการเชิงประจักษ์ (Empirical formula) สำหรับพื้นที่ชายฝั่งทะเล


**Accuracy dependent on your coefficient!!!**


Thepchai Srinoi and Associate Professor Dr. Phisan Santitamnont

Department of Survey Engineering

Faculty of Engineering

Chulalongkorn University

*ผู้ใช้งานจัดหาค่าสัมประสิทธิ์ของ Stumpf Algorithm *

แรงบันดาลใจการพัฒนาโปรแกรม/Additional reading for improve this code ...

https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/satellite_derived_bathymetry_mapping-sdbm/

Prepared your library ....

In [ ]:
import os
import ee
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import folium as fo
import folium
import math as m
import math

Authenticate to GEE servers .... Generate Token first !!

In [ ]:
# Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=3uiZRF2z7ROvTCoXaxyD8QixT67J81pAT-rZZqWyfKk&tc=5lY-J1HhGPgCkJ8vmMLqaCwYbClwYjBv40Gv9AsgRtM&cc=lvSsLgYV8O9KboIxGLAMlEdrl7nVVE8fAgoiDI0rUaU

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjsLKQStjfXZNmSFzQdVT8GVQ5yniaup8DnRe8C9IpZ5_CInSzTTfM

Successfully saved authorization token.


In [ ]:
# Install Function for Cloud Mask
def maskS2clouds(image) :
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0)) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Map Visualization : Google Earth Engine
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

Select your area of interest, date, band 8 threshold for landmask

In [ ]:
####################################### OVERVIEW SATELLITE IMAGE

lat_min, lat_max, lon_min, lon_max = (12.498140, 12.715250, 101.074208, 101.966847)  ## <--- พื้นที่ทำงานของเรา แนะให้ตีจาก code.earthengine.google.com 
AOI = ee.Geometry.Polygon(
        [[[lon_min, lat_max],
           [lon_min, lat_min],
           [lon_max, lat_min],
           [lon_max, lat_max]]])

fromdate = '2021-1-1'     # <----- วันที่เริ่มค้นภาพ
todate = '2022-1-1'       # <----- วันที่สิ้นสุดค้นภาพ  เลือกช่วงเวลาที่คาดว่าไม่มีเมฆปกคลุม และได้ภาพที่ถ่ายได้เห็นความลึกดีๆ

image = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(fromdate,todate).\
          filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',8).\   # <---- ร้อยละการปกคลุมเมฆ ผู้ใช้ต้องเลือกเติมตัวเลขเอง
          filterBounds(AOI).map(maskS2clouds).mosaic().clip(AOI)      

### LANDMASK BY BAND 8 THRESHOLD
B8_threshold = 0.04     # <----- การใส่หน้ากากให้บริเวณที่คาดว่าเป็นดิน ผู้ใช้ต้องเลือกเติมตัวเลขเอง

B8_c = (image.select('B8').lt(B8_threshold)).selfMask()
B2_lm = image.select('B2').multiply(B8_c)
B3_lm = image.select('B3').multiply(B8_c)
B4_lm = image.select('B4').multiply(B8_c)
B8_lm = image.select('B8').multiply(B8_c)
image = image.addBands(B2_lm.rename('B2_lm'))
image = image.addBands(B3_lm.rename('B3_lm'))
image = image.addBands(B4_lm.rename('B4_lm'))
image = image.addBands(B4_lm.rename('B8_lm'))

############### VISUALIZATION ############################## ส่วนของการแสดงผลภาพถ่ายดาวเทียม
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=11)

map_matched.add_ee_layer(image,{'min':0, 'max':0.3, 'bands':['B4','B3','B2']},'RGB_L2A')
map_matched.add_ee_layer(image,{'min':0, 'max':0.3, 'bands':['B4_lm','B3_lm','B2_lm']},'RGB_L2A_landmask')

display(map_matched.add_child(folium.LayerControl()))

Do the bathymetry calculation, please select your suitable coefficient ...

In [ ]:
#### SATELLITE DERIVED BATHYMETRY ALGORITHM - STUMPF ALGORITHM

if 1:   # Open/Close SDB Algorithm
  A = (image.select('B2_lm')).multiply(1000)
  B = (image.select('B3_lm')).multiply(1000)
  B2B3 = (A.log()).divide(B.log())
  #SDB_ST = m0 + m1((ln1000B2)/(ln1000B3)) Stumpf Algorithm
  image = image.addBands(B2B3.rename('B2B3'))

  # Use Coefficient from Koh Mark Trat : SDB Senior Project ...  ตัวนี้สำคัญ ค่าสัมประสิทธิ์อัลกอริทึมที่ต้องจัดหามาคำนวณแปลงเป็นความลึก
  model_sdb_coef_ = 68.1061068420549
  model_sdb_intercept_ = -63.72996441

  SDB_ST = ((image.select('B2B3')).multiply(float(model_sdb_coef_))).add(model_sdb_intercept_)
  image = image.addBands(SDB_ST.rename('SDB_ST'))
  print('Stumpf finish ....')

############### VISUALIZATION ############################## 
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=11)

depth_palette = ['#FE1B09','#FEB009', '#FEFB09','#B8FE09', '#09FEF3', '#0927FE']

map_matched.add_ee_layer(image,{'min':0, 'max':0.3, 'bands':['B4','B3','B2']},'RGB_L2A')
map_matched.add_ee_layer(image.select('SDB_ST'),{'min': 0, 'max': 16, 'palette' : depth_palette},'SDB Stumpf') 

display(map_matched.add_child(folium.LayerControl()))

Stumpf finish ....


export geotiff to your google drive ...

In [ ]:
lat_min, lat_max, lon_min, lon_max = (12.498140, 12.715250, 101.074208, 101.966847)  ## <--- พื้นที่ทำงานของเรา แนะให้ตีจาก code.earthengine.google.com 
AOI = ee.Geometry.Polygon(
        [[[lon_min, lat_max],
           [lon_min, lat_min],
           [lon_max, lat_min],
           [lon_max, lat_max]]])

sdb_band = image.select('SDB_ST')               # <--- เลือกแบนด์ที่ต้องการส่งออก
task = ee.batch.Export.image.toDrive(**{
    'image': sdb_band,                          # <--- ชื่อตัวแปร raster from google earth engine 
    'description': 'sdb_rayongtochantaburi',    # <--- ชื่อไฟล์ภาพนำออก .tiff
    'folder':'mybathymetry',                    # <--- folder เก็บไฟล์
    'scale': 10,                                # <--- spatial resolution
    'region': AOI.getInfo()['coordinates']      # <--- บริเวณพื้นที่งานที่จะส่งออก
})
task.start()

import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id: AFN3MRENRYPRP22JOUMTRNBE).
Polling for task (id